In [1]:
import datetime
import os
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from glob import glob
from pprint import pprint
from pymongo import MongoClient
from time import sleep
from zeep import Client

**1). Модифицировать приложение из предыдущего домашнего задания:**
 - приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru
 - собранные данные должны быть приведены к общей структуре
 - разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям.

**2). Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)**

**3). Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.**

**4). При наличии вакансии с зарплатой отличной от руб. приложение должно осущетсвлять автоматический перевод по курсу валюты на сегодняшний день. Информацию получать через SOAP-сервис центробанка.**

In [2]:
columns = ['vacancy', 'snippet', 'min_salary', 'max_salary', 'currency', 'employer', 'site', 'link']
hh = pd.DataFrame(columns=columns)
sj = pd.DataFrame(columns=columns)

Получим wsdl от ЦБ:

In [3]:
wsdl = 'http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
zeep_client = Client(wsdl)

Функция получения текущего курса задаваемой валюты:

In [4]:
def get_current_rate(currency_name='Доллар США'):
    now_date = datetime.datetime.now()
    rates_on_date = zeep_client.service.GetCursOnDate(now_date)
    for currency in rates_on_date._value_1._value_1:
        for _, currency_info in currency.items():
            if currency_name in currency_info['Vname']:
                current_rate = float(currency_info['Vcurs'])
    return current_rate

Подключимся к БД:

In [5]:
client = MongoClient('mongodb://127.0.0.1:27017')

db_name = 'test'

Если такая БД есть, пересоздаем ее:

In [6]:
db_list = client.list_database_names()
if db_name in db_list:
    client.drop_database(db_name)

In [7]:
db = client[db_name]
docs = db.docs

Функция с проверкой, есть ли вакансия в БД - если нет, то добавляем вакансию:

In [8]:
def add_to_db(db, vacancy_info):
    is_link_in = db.find_one({'link': vacancy_info['link']})
    if is_link_in == None:
        result = db.insert_one(vacancy_info)

Через парсинг сохраненных страниц сайта hh.ru получим данные по вакансиям (добавим их в датафрейм и БД):

In [9]:
for file_name in glob(os.path.join('hh', '*.html')):
    with open(file_name, 'rb') as f:
        parser = bs(f, 'html.parser')
        vacancies = parser.find_all('div', {'class': 'vacancy-serp-item'})
        for v in vacancies:
            vacancy = v.find('a', {'class': 'bloko-link HH-LinkModifier'}, href=True)
            snippet = v.find('div', {'data-qa': "vacancy-serp__vacancy_snippet_requirement"})
            snippet = ''.join(snippet.get_text())
            salary_info = v.find('div', {'class': "vacancy-serp-item__compensation"})
            if salary_info:
                salary_text = salary_info.get_text()
                salary_text = salary_text.replace('\xa0', '')
                salaries = re.findall('[0-9]+', salary_text)
                min_salary = int(salaries[0])
                if len(salaries) == 2:
                    max_salary = int(salaries[1])
                else:
                    max_salary = np.nan
                currency_info = re.findall('руб|USD', salary_text)
                if 'руб' in currency_info:
                    currency = 'руб.'
                elif 'USD' in currency_info:
                    currency = 'руб.'
                    current_rate = get_current_rate()
                    min_salary *= current_rate
                    max_salary *= current_rate
                else:
                    currency = 'у.е.'
            else:
                min_salary = np.nan
                max_salary = np.nan
                currency = np.nan
            employer = v.find('a', {'data-qa': "vacancy-serp__vacancy-employer"})
            link = vacancy['href']
            link = re.sub('[?]query=.+', '', link)            
            vacancy_info = {
                'vacancy': vacancy.get_text(),
                'snippet': snippet,
                'currency': currency,
                'min_salary': min_salary,
                'max_salary': max_salary,
                'employer': employer.get_text(),
                'site': 'hh',
                'link': link,
            }
            hh = hh.append(vacancy_info, ignore_index=True)
            add_to_db(docs, vacancy_info)

In [10]:
hh.drop_duplicates(inplace=True)

hh.head()

,vacancy,snippet,min_salary,max_salary,currency,employer,site,link
0,Программист С++,Отличное знание C++. Умение разбираться в чужо...,60000,100000,руб.,Cappasity Inc.,hh,https://hh.ru/vacancy/32323917
1,Программист С++/Qt,"QT framework. Знание паттернов, разработка мно...",60000,100000,руб.,Cappasity Inc.,hh,https://hh.ru/vacancy/31273030
2,Full-Stack Разработчик JavaScript (Node.js + A...,JavaScript / TypeScript (идеальное владение). ...,120000,NaN,руб.,ИП Бушев Юрий Владимирович,hh,https://hh.ru/vacancy/32859767
3,Программист (C#\ ASP.NET\MVC),Высшее профильное образование. Опыт разработки...,100000,NaN,руб.,ООО МИЮИ,hh,https://hh.ru/vacancy/33036570
4,Web-программист/PHP-программист,"Знание PHP, MySQL‚ HTML‚ CSS‚ JavaScript. - Оп...",NaN,NaN,NaN,Онлишар,hh,https://hh.ru/vacancy/32728996


Через запросы на сайт superjob.ru и парсинг результатов запроса получим данные по вакансиям (добавим их в датафрейм и БД):

In [11]:
for page in ['', '?page=3', '?page=7']:
    url = 'https://www.superjob.ru/vakansii/programmist.html' + page
    r = requests.get(url)
    print(r)
    sleep(6)
    parser = bs(r.text, 'html.parser')
    vacancies = parser.find_all('div', {'class': '_3zucV _2GPIV i6-sc _3VcZr'})
    for v in vacancies:
        vacancy = v.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
        salary_text = v.find('span', {'class': 'f-test-text-company-item-salary'}).get_text()
        if salary_text == 'По договорённости':
            min_salary = np.nan
            max_salary = np.nan
            currency = np.nan
        else:
            salary_text = salary_text.replace('\xa0', '')
            salaries = re.findall('[0-9]+', salary_text)
            min_salary = int(salaries[0])
            if len(salaries) == 2:
                max_salary = int(salaries[1])
            else:
                max_salary = np.nan
            currency = 'руб.'
        employer_info = v.find('span', {'class': 'f-test-text-vacancy-item-company-name'})
        if employer_info:
            employer = employer_info.get_text()
        else:
            employer = np.nan
        snippet_info = v.find_all('span', {'class': '_3mfro _9fXTd _2JVkc _2VHxz'})
        if len(snippet_info) == 2:
            snippet = snippet_info[0].get_text().replace('Должностные обязанности: ', '')
            snippet += snippet_info[1].get_text().replace('Требования:', '')
        else:
            snippet = np.nan
        link = v.find('a', {'class': '_1QIBo'}, href=True)['href']
        link = 'https://www.superjob.ru' + link
        vacancy_info = {
                    'vacancy': vacancy.get_text(),
                    'snippet': snippet,
                    'currency': currency,
                    'min_salary': min_salary,
                    'max_salary': max_salary,
                    'employer': employer,
                    'site': 'sj',
                    'link': link,
                }
        sj = sj.append(vacancy_info, ignore_index=True)
        add_to_db(docs, vacancy_info)

<Response [200]>
<Response [200]>
<Response [200]>


In [12]:
sj.drop_duplicates(inplace=True)

sj.head()

,vacancy,snippet,min_salary,max_salary,currency,employer,site,link
0,Ведущий программист,Проведение обследования и постановка задач на ...,49000,60000,руб.,Российская детская клиническая больница Минздр...,sj,https://www.superjob.ru/vakansii/veduschij-pro...
1,Web-программист,Центр по разработке ПО в сфере логистики пригл...,150000,NaN,руб.,Кадровое агентство уникальных специалистов,sj,https://www.superjob.ru/vakansii/web-programmi...
2,Программист 1С,Разработка и внедрение программного продукта в...,100000,NaN,руб.,Группа компаний ЦВТ,sj,https://www.superjob.ru/vakansii/programmist-1...
3,Программист PHP,Разработка и сопровождение WEB-сервисов предпр...,NaN,NaN,NaN,Национальное агентство клинической фармакологи...,sj,https://www.superjob.ru/vakansii/programmist-p...
4,Инженер-программист,Проектирование систем обработки информации. От...,NaN,NaN,NaN,"""Корпорация ""ВНИИЭМ""",sj,https://www.superjob.ru/vakansii/inzhener-prog...


Объединим в датафрейм полученные данные с hh.ru и superjob.ru:

In [13]:
df = pd.concat([hh, sj], sort=False, ignore_index=True)

Напишем функцию, которая фильтрует записи датафрейма по названию языка в столбцах с названием и описанием вакансии.
Сортируем их по зарплате - по убыванию:

In [14]:
def filter_vacancies(l, df):
    idx = df['vacancy'].str.contains(l, regex=False) | \
          df['snippet'].str.contains(l, regex=False)
    return df[idx].sort_values(by=['min_salary', 'max_salary'], ascending=False)

Отфильтруем в датафрейме вакансии по С++:

In [15]:
#filter_vacancies('C++', df=df)

Напишем функцию, выводящую из БД вакансии с зарплатой более заданного значения:

In [16]:
def filter_salary(db, salary=200000):
    results = db.find({"min_salary": {'$gte': salary}})
    for r in results:
        pprint(r)

In [17]:
filter_salary(docs, salary=200000)

{'_id': ObjectId('5d5fb7ff7928b25c7393b8b7'),
 'currency': 'руб.',
 'employer': ' HeadHunter::Технический департамент',
 'link': 'https://hh.ru/vacancy/32144172',
 'max_salary': nan,
 'min_salary': 200000,
 'site': 'hh',
 'snippet': 'Очень хорошо знает JavaScript и React. Понимает на что нужно '
            'обращать внимание при разработке страниц. Командный игрок и хочет '
            'обсуждать...',
 'vacancy': 'Senior Front-end Developer (JavaScript, React)'}
{'_id': ObjectId('5d5fb7ff7928b25c7393b8c0'),
 'currency': 'руб.',
 'employer': ' HeadHunter::Технический департамент',
 'link': 'https://hh.ru/vacancy/32893126',
 'max_salary': nan,
 'min_salary': 200000,
 'site': 'hh',
 'snippet': 'Хорошее владение Java и опыт объектно-ориентированного '
            'программирования. Знание основных библиотек и шаблонов '
            'проектирования. Знание основных алгоритмов и структур данных. ',
 'vacancy': 'Senior java developer'}
{'_id': ObjectId('5d5fb7ff7928b25c7393b8c2'),
 'currency

Выведем из датафрейма вакансии с минимаьной зарплатой более 200 т.р. для проверки:

In [18]:
df[df['min_salary'] >= 200000] 

,vacancy,snippet,min_salary,max_salary,currency,employer,site,link
12,"Senior Front-end Developer (JavaScript, React)",Очень хорошо знает JavaScript и React. Понимае...,200000,NaN,руб.,HeadHunter::Технический департамент,hh,https://hh.ru/vacancy/32144172
21,Senior java developer,Хорошее владение Java и опыт объектно-ориентир...,200000,NaN,руб.,HeadHunter::Технический департамент,hh,https://hh.ru/vacancy/32893126
23,Senior Software Developer (new product for QA),...interests if you: Want to develop in Kotlin...,200000,NaN,руб.,JetBrains,hh,https://hh.ru/vacancy/31891023
44,Старший программист-разработчик PHP,Отличное знание MySQL на уровне разработчика. ...,200000,200000,руб.,ООО Матрас.ру,hh,https://hh.ru/vacancy/32946587
50,Backend разработчик Java (ДИТ HR),Опыт от 2-х лет разработки систем с трудозатра...,230000,NaN,руб.,Сбербанк для экспертов,hh,https://hh.ru/vacancy/32377870
52,JavaScript разработчик (ДИТ HR),"JS на продвинутом уровне: closures, lexical en...",230000,NaN,руб.,Сбербанк для экспертов,hh,https://hh.ru/vacancy/32379189
103,Ведущий IOS программист,Опыт работы от 3 лет. Знание языка программиро...,300000,NaN,руб.,ООО 5Вижинс,hh,https://hh.ru/vacancy/31066860
